In [1]:
import pandas as pd

In [2]:
!pwd

/Users/Rrando/Documents/GitHub/content-ml-experiments/tab_grouping


In [1]:
fine_tune_raw = pd.read_csv("./extract/guided_output_2_10.csv")

NameError: name 'pd' is not defined

In [4]:
fine_tune_raw.sample(n=100)

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,input_titles,input_keywords,output,assigned_topic,Hint,hint_guided_output
2507,2507,1028,1028,Collectors Community Events Calendar\nUpcoming...,"collectors,upcoming,webinars",Collector Events and Webinars,92,NaN,Collector Events
850,850,2450,2450,High-mileage electric car usage is working out...,NaN,Electric Vehicle Costs,16,NaN,Electric Vehicle Costs
976,976,1379,1379,Vintage Handmade Wooden Toy - Etsy\nCustom Han...,"etsy,handmade,items",Handmade Items on Etsy,20,NaN,Handmade Items on Etsy
2088,2088,2221,2221,Why the Past 10 Years of American Life Have Be...,NaN,American Society Analysis,62,NaN,Cultural Commentary
812,812,2403,2403,How To Turn One LinkedIn Post Into A Week Of C...,NaN,Content Creation Tips,15,NaN,Content Creation
...,...,...,...,...,...,...,...,...,...
2042,2042,1932,1932,"Farewell, then, Justin Welby. Good to see that...",NaN,Opinion Piece,60,NaN,Opinion Column
2438,2438,2033,2033,The JBL PartyBox Stage 320 Speaker Is Still $2...,NaN,Shopping Deal,85,NaN,Tech Deals
100,100,309,309,Online Bank - Payment History\nOnline Bank - C...,"bank,financial,management",Financial Services,1,NaN,Financial Management
1102,1102,266,266,Alternate History Forum\nRPG Historical Gamers...,"historical,alternate,discussion",Historical Discussions & Alternate History,24,NaN,Alternate History Discussion


In [5]:
fine_tune_raw.hint_guided_output.str.len().mean()

17.55394272263633

In [6]:
fine_tune_raw.output.str.len().mean()

19.29109454688113

In [9]:
len(fine_tune_raw[fine_tune_raw.hint_guided_output.str.contains("&")].output.to_list())

143

In [10]:
len(fine_tune_raw[fine_tune_raw.output.str.contains(" & ")].output.to_list())

260

In [11]:
len(fine_tune_raw[fine_tune_raw.hint_guided_output.str.contains(" and ")].output.to_list())

73

In [13]:
len(fine_tune_raw[fine_tune_raw.output.str.contains(" and ")].output.to_list())

104

In [14]:
fine_tune_raw[fine_tune_raw.output.str.contains(" or ")].output.to_list()

[]

In [15]:
def remove_and(a: str):
    words = a.split(" ")
    if len(words) < 2:
        return a
    for stopword in ["and", "&"]:
        if stopword in words:
            and_pos = words.index(stopword)
            if and_pos > 0:
                return " ".join(words[0:and_pos])
    return " ".join(words)


In [27]:
remove_and("Word & Dog")

'Word'

In [16]:
def remove_or(a: str):
    words = a.split(" ")
    if len(words) < 3:
        return a
    if words[-2] == 'or':
        words = words[:-2]
    return " ".join(words)


In [17]:
def remove_news(a: str):
    words = a.split(" ")
    if len(words) == 1:
        return a
    if words[-1] == 'News':
        words = words[:-1]
    return " ".join(words)


In [18]:
remove_news("Tech News")

'Tech'

In [19]:
def remove_year(a: str):
    words = list(filter(lambda word: word not in ["2023", "2024"], a.split(" ")))
    return " ".join(words)


In [18]:
remove_or("News or Reviews")

'News'

In [19]:
remove_year("Review 2023 News")

'Review News'

In [20]:
fine_tune_raw["output_orig"] = fine_tune_raw["output"]

In [21]:
fine_tune_raw["output"] = fine_tune_raw["hint_guided_output"].apply(remove_and)

In [22]:
fine_tune_raw["output"] = fine_tune_raw["output"].apply(remove_or)

In [23]:
fine_tune_raw["output"] = fine_tune_raw["output"].apply(remove_year)

In [24]:
fine_tune_raw.sample(n=10)

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,input_titles,input_keywords,output,assigned_topic,Hint,hint_guided_output,output_orig
1333,1333,904,904,Educational Streams: Festivals and Awards Arou...,NaN,Cultural Education,31,NaN,Cultural Education,Global Festivals Education
1224,1224,84,84,Workspace | Slack\nData Analysis Course | Cour...,"analysis,style,student",Student Resources,28,NaN,Student Resources,Student Tools
438,438,2467,2467,Rivalry games emotional enough without plantin...,NaN,Sports Commentary,6,NaN,Sports Commentary,Sportsmanship in Rivalry Games
1479,1479,1309,1309,Top Online Business Tools - Forbes\nStartups a...,"startups,startup,advice",Startup Resources,36,NaN,Startup Resources,Startup Resources
2495,2495,2481,2481,This Week’s Sex Story: A Woman Who Talks About...,NaN,Relationships,90,NaN,Relationships,Sex Stories
1422,1422,1033,1033,Drones in Modern Warfare: An Overview\nWWII De...,"defense,air,warfare",Military Defense,34,NaN,Military Defense,Military Defense
224,224,1439,1439,Best Restaurants in Copenhagen - TripAdvisor\n...,"copenhagen,airbnb,unique",Copenhagen Travel,2,NaN,Copenhagen Travel,Copenhagen Travel
2264,2264,1225,1225,Wedding Inspirations - TikTok\nVendor Meetups ...,"inspirations,meetups,pinterest",Event Inspirations,72,NaN,Event Inspirations,Event Inspirations
287,287,65,65,Questions About 2023 Winter Forecast - Weather...,"weather,veterans,2023",Weather Preparedness,4,Weather,2023 Weather Preparedness,2023 Weather Preparedness
1196,1196,373,373,Unique Shopping Experiences in San Francisco\n...,"california,blogs,unique",California Fashion,27,NaN,California Fashion & Lifestyle,California Lifestyle


In [23]:
# fine_tune_raw["output_no_news"] = fine_tune_raw["output_no_year"].apply(remove_news)

In [59]:
#fine_tune_raw["output"] = fine_tune_raw["output"].apply(remove_or)

In [25]:
fine_tune_raw[fine_tune_raw.output.str.contains("&")].output.to_list()

[]

In [26]:
fine_tune_raw[fine_tune_raw.output.str.contains("and")].output.to_list()

['Galápagos Islands Travel',
 'Kathmandu Travel',
 'Kathmandu Highlights',
 'Standardized Testing',
 'Standardized Testing',
 'Handmade Crafts',
 'Handmade Items on Etsy',
 'Political Scandal',
 'Political Scandals',
 'Political Scandals',
 'Political Scandals',
 'Digital Branding',
 'Digital Branding',
 'Anime Merchandise',
 'Anime Merchandise',
 'Mozilla Branding News',
 'Hand Sanitizers',
 'Pandemic History',
 'Pandemic Business Strategies',
 'Pandemic Insights',
 'Pandemic History']

In [27]:
fine_tune_raw.to_csv("./test_data/topic_fine_tuning_data_guided__02_11_processed.csv")